## Failed Approach (Using API)

The following blocks try to use the API approach, which failed fantastically. I suggest you go to the next section, which works well.

You can also use directly PushishiftAPI() without psaw.

```Python
from pushshift_py import PushshiftAPI
import datetime as dt
import psaw
import pandas as pd
import requests
import json
import csv
import time
api = psaw.PushshiftAPI()

startEpoch = int(dt.datetime(2020,1,1).timestamp())
```
    
The following block shows how we can get information using pushshift. It shows how we can specify the features and get them. The returned data type is a generator with "submission" type as elements, though we can certainly make them into a list.

```Python
features = ['url','author', 'title', 'subreddit', 'id', 'created', 'score']
subreddit = 'NBA'

data = api.search_submissions(after=startEpoch,
                            subreddit=subreddit,
                            filter= features,
                            limit=10)

for datum in data:
    print(datum.id, datum.subreddit, datum.title, datum.author, datum.url, datum.created, datum.score)

import praw

reddit = praw.Reddit(
    client_id="kxbUr-4PyE7DlQ",
    client_secret="Q5rIAPS9IHZ1QgOIkHNY09Y9VMxDsA",
    password="AACAXZDE",
    user_agent="testscript by u/kc_the_scraper",
    username="kc_the_scraper",
)
```

We can use praw to get the post body using the following block.
```Python
reddit.submission(id='eiev5d').selftext
```



In the following blocks, we create tables and store the information. For some reason, though, the api often acts up and freezes when we loop through the data.
```Python
import sqlite3

conn = sqlite3.connect('redditPosts.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Posts(
                id TEXT PRIMARY KEY,
                subreddit TEXT,
                title TEXT,
                author TEXT,
                url TEXT,
                created int)
                ''')

features = ['url','author', 'title', 'subreddit', 'id', 'created']
subreddit = 'stocks'
latest = dt.datetime(2021,5,8).timestamp()
earliest = dt.datetime(2020,1,1).timestamp()

startEpoch = earliest

while startEpoch <= latest:
    data = api.search_submissions(after=startEpoch,
                            subreddit=subreddit,
                            filter= features,
                            limit=100)
    
    for datum in data:
        print('Got here 2.')
        cur.execute('''INSERT OR IGNORE INTO Posts VALUES (?,?,?,?,?,?)'''
                    , (datum.id, datum.subreddit, datum.title, datum.author, datum.url, datum.created))
        
        currentTime = datum.created
    
    conn.commit()
    if currentTime == startEpoch:
        break
    startEpoch = currentTime + 1
    print(dt.datetime.fromtimestamp(startEpoch))    

```

## Another Approach (Getting JSON)

The method above is shaky at best. A lot of times the api just freezes. On the other hand, I find using requests much easier. The following code blocks contain what you need for storing reddit data you need.

In [1]:
import requests
import datetime as dt
import sqlite3
import json
import time
import sys

In [2]:
def getPushShiftData(after,before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?size=100&after='+str(int(after))+'&before='+str(int(before))+'&subreddit='+str(sub)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

def extractInfo(datum,features):
    info = {}
    
    for feature in features:
        info[feature] = datum[feature]
    
    return info

def getLatestTime(data):
    return data[-1]['created_utc']

def dataStoragePipeline(after, before, sub, conn):
    features = ['full_link','author', 'title', 'subreddit', 'id', 'created_utc']
    cursor = conn.cursor()
    while after < before:
        data = getPushShiftData(after, before, sub)
        if not data:
            print("There is no data anymore.")
            return 1
        for datum in data:
            cursor.execute('''INSERT OR IGNORE INTO Posts 
                                VALUES (?,?,?,?,?,?)'''
                              , (datum['id'], datum['subreddit'], datum['title'], datum['author'], datum['full_link'], datum['created_utc']))
        
        after = getLatestTime(data) + 1
        conn.commit()
        print("The latest post is submitted at", dt.datetime.fromtimestamp(after-1))
        time.sleep(0.1)
    
        
        

In [3]:
conn = sqlite3.connect('redditPosts.sqlite')
cur = conn.cursor()
subreddit = 'dogecoin'
end = int(time.time()-86400) #I subtracted by one day, so that we have some buffer.
start = dt.datetime(2021,1,1).timestamp()
cur.execute('''SELECT MIN(created), MAX(created) FROM Posts
                WHERE subreddit = ?''', (subreddit,))
datatimes = cur.fetchone()

if datatimes[0]:
    dataEarly, dataLate = datatimes
    if end < dataEarly:
        end = dataEarly
    elif start < dataLate:
        start =dataLate

In [33]:
while start < end:
    try:
        flag = dataStoragePipeline(after = start, before = end, sub = subreddit, conn = conn)
        if flag:
            break
    except KeyboardInterrupt:
        print("Interrupted by keyboard. Stopping.")
        break
        
    except:
        print("Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.")
        time.sleep(1)
        cur.execute('''SELECT MIN(created), MAX(created) FROM Posts
                        WHERE subreddit = ?''', (subreddit,))
        datatimes = cur.fetchone()
        
        if datatimes[0]:
            dataEarly, dataLate = datatimes
            if end < dataEarly:
                end = dataEarly
            elif start < dataLate:
                start =dataLate
        

The latest post is submitted at 2021-01-01 18:30:03
The latest post is submitted at 2021-01-01 22:35:02
The latest post is submitted at 2021-01-02 00:02:23
The latest post is submitted at 2021-01-02 07:48:57
The latest post is submitted at 2021-01-02 09:50:20
The latest post is submitted at 2021-01-02 11:09:15
The latest post is submitted at 2021-01-02 12:10:58
The latest post is submitted at 2021-01-02 12:57:15
The latest post is submitted at 2021-01-02 13:42:57
The latest post is submitted at 2021-01-02 14:49:28
The latest post is submitted at 2021-01-02 16:44:42
The latest post is submitted at 2021-01-02 19:55:39
The latest post is submitted at 2021-01-02 22:45:04
The latest post is submitted at 2021-01-03 00:59:35
The latest post is submitted at 2021-01-03 06:49:31
The latest post is submitted at 2021-01-03 10:19:34
The latest post is submitted at 2021-01-03 12:28:51
The latest post is submitted at 2021-01-03 16:02:46
The latest post is submitted at 2021-01-03 19:43:45
The latest p

The latest post is submitted at 2021-01-28 17:30:12
The latest post is submitted at 2021-01-28 17:36:55
The latest post is submitted at 2021-01-28 17:39:54
The latest post is submitted at 2021-01-28 17:41:50
The latest post is submitted at 2021-01-28 17:44:45
The latest post is submitted at 2021-01-28 17:47:41
The latest post is submitted at 2021-01-28 17:50:05
The latest post is submitted at 2021-01-28 17:53:31
The latest post is submitted at 2021-01-28 17:57:04
The latest post is submitted at 2021-01-28 18:00:33
The latest post is submitted at 2021-01-28 18:03:07
The latest post is submitted at 2021-01-28 18:06:04
The latest post is submitted at 2021-01-28 18:09:30
The latest post is submitted at 2021-01-28 18:13:05
The latest post is submitted at 2021-01-28 18:16:46
The latest post is submitted at 2021-01-28 18:19:24
The latest post is submitted at 2021-01-28 18:22:35
The latest post is submitted at 2021-01-28 18:24:55
The latest post is submitted at 2021-01-28 18:26:48
The latest p

The latest post is submitted at 2021-01-28 22:31:54
The latest post is submitted at 2021-01-28 22:34:05
The latest post is submitted at 2021-01-28 22:35:48
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-28 22:37:54
The latest post is submitted at 2021-01-28 22:39:52
The latest post is submitted at 2021-01-28 22:41:57
The latest post is submitted at 2021-01-28 22:44:30
The latest post is submitted at 2021-01-28 22:46:49
The latest post is submitted at 2021-01-28 22:49:16
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-28 22:51:08
The latest post is submitted at 2021-01-28 22:53:02
The latest post is submitted at 2021-01-28 22:54:56
The latest post is submitted at 2021-01-28 22:57:17
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-28 22:59:20
The latest post i

The latest post is submitted at 2021-01-29 05:44:05
The latest post is submitted at 2021-01-29 05:49:11
The latest post is submitted at 2021-01-29 05:53:48
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 05:58:31
The latest post is submitted at 2021-01-29 06:02:40
The latest post is submitted at 2021-01-29 06:06:54
The latest post is submitted at 2021-01-29 06:10:54
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 06:16:04
The latest post is submitted at 2021-01-29 06:20:29
The latest post is submitted at 2021-01-29 06:24:14
The latest post is submitted at 2021-01-29 06:28:15
The latest post is submitted at 2021-01-29 06:31:22
The latest post is submitted at 2021-01-29 06:35:05
The latest post is submitted at 2021-01-29 06:37:48
The latest post is submitted at 2021-01-29 06:40:40
The latest post is submitted at 2021-01-29 06:43:4

The latest post is submitted at 2021-01-29 12:06:20
The latest post is submitted at 2021-01-29 12:08:40
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 12:11:13
The latest post is submitted at 2021-01-29 12:13:57
The latest post is submitted at 2021-01-29 12:17:12
The latest post is submitted at 2021-01-29 12:19:58
The latest post is submitted at 2021-01-29 12:22:35
The latest post is submitted at 2021-01-29 12:24:55
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 12:27:25
The latest post is submitted at 2021-01-29 12:30:05
The latest post is submitted at 2021-01-29 12:32:30
The latest post is submitted at 2021-01-29 12:34:42
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 12:36:48
The latest post is submitted at 2021-01-29 12:39:16
The latest post i

...

The latest post is submitted at 2021-05-05 13:53:12
The latest post is submitted at 2021-05-05 14:02:02
The latest post is submitted at 2021-05-05 14:10:39
The latest post is submitted at 2021-05-05 14:19:25
The latest post is submitted at 2021-05-05 14:29:53
The latest post is submitted at 2021-05-05 14:39:54
The latest post is submitted at 2021-05-05 14:50:03
The latest post is submitted at 2021-05-05 14:59:49
The latest post is submitted at 2021-05-05 15:09:52
The latest post is submitted at 2021-05-05 15:19:04
The latest post is submitted at 2021-05-05 15:28:30
The latest post is submitted at 2021-05-05 15:35:16
The latest post is submitted at 2021-05-05 15:43:13
The latest post is submitted at 2021-05-05 15:51:46
The latest post is submitted at 2021-05-05 16:00:43
The latest post is submitted at 2021-05-05 16:10:45
The latest post is submitted at 2021-05-05 16:20:41
The latest post is submitted at 2021-05-05 16:29:29
The latest post is submitted at 2021-05-05 16:38:59
The latest p

The latest post is submitted at 2021-05-06 21:00:44
The latest post is submitted at 2021-05-06 21:07:52
The latest post is submitted at 2021-05-06 21:12:57
The latest post is submitted at 2021-05-06 21:17:03
The latest post is submitted at 2021-05-06 21:24:05
The latest post is submitted at 2021-05-06 21:31:20
The latest post is submitted at 2021-05-06 21:37:53
The latest post is submitted at 2021-05-06 21:45:00
The latest post is submitted at 2021-05-06 21:52:04
The latest post is submitted at 2021-05-06 22:00:08
The latest post is submitted at 2021-05-06 22:07:50
The latest post is submitted at 2021-05-06 22:14:45
The latest post is submitted at 2021-05-06 22:22:38
The latest post is submitted at 2021-05-06 22:30:11
The latest post is submitted at 2021-05-06 22:37:48
The latest post is submitted at 2021-05-06 22:47:51
The latest post is submitted at 2021-05-06 22:58:38
The latest post is submitted at 2021-05-06 23:11:12
The latest post is submitted at 2021-05-06 23:25:45
The latest p

The latest post is submitted at 2021-05-08 00:21:51
The latest post is submitted at 2021-05-08 00:33:43
The latest post is submitted at 2021-05-08 00:45:32
The latest post is submitted at 2021-05-08 00:55:51
The latest post is submitted at 2021-05-08 01:07:29
The latest post is submitted at 2021-05-08 01:23:15
The latest post is submitted at 2021-05-08 01:36:28
The latest post is submitted at 2021-05-08 01:51:47
The latest post is submitted at 2021-05-08 02:08:08
The latest post is submitted at 2021-05-08 02:22:37
The latest post is submitted at 2021-05-08 02:40:13
The latest post is submitted at 2021-05-08 02:59:33
The latest post is submitted at 2021-05-08 03:20:02
The latest post is submitted at 2021-05-08 03:41:10
The latest post is submitted at 2021-05-08 04:05:26
The latest post is submitted at 2021-05-08 04:30:27
The latest post is submitted at 2021-05-08 04:52:13
The latest post is submitted at 2021-05-08 05:14:07
The latest post is submitted at 2021-05-08 05:29:52
The latest p

The latest post is submitted at 2021-05-08 22:29:21
The latest post is submitted at 2021-05-08 22:31:35
The latest post is submitted at 2021-05-08 22:33:39
The latest post is submitted at 2021-05-08 22:35:28
The latest post is submitted at 2021-05-08 22:36:48
The latest post is submitted at 2021-05-08 22:38:29
The latest post is submitted at 2021-05-08 22:40:22
The latest post is submitted at 2021-05-08 22:43:51
The latest post is submitted at 2021-05-08 22:44:57
The latest post is submitted at 2021-05-08 22:45:37
The latest post is submitted at 2021-05-08 22:46:34
The latest post is submitted at 2021-05-08 22:47:27
The latest post is submitted at 2021-05-08 22:48:59
The latest post is submitted at 2021-05-08 22:51:04
The latest post is submitted at 2021-05-08 22:52:46
The latest post is submitted at 2021-05-08 22:53:51
The latest post is submitted at 2021-05-08 22:54:59
The latest post is submitted at 2021-05-08 22:55:50
The latest post is submitted at 2021-05-08 22:56:42
The latest p

The latest post is submitted at 2021-05-09 06:36:20
The latest post is submitted at 2021-05-09 06:42:20
The latest post is submitted at 2021-05-09 06:48:27
The latest post is submitted at 2021-05-09 06:55:07
The latest post is submitted at 2021-05-09 06:59:50
The latest post is submitted at 2021-05-09 07:05:08
The latest post is submitted at 2021-05-09 07:08:12
The latest post is submitted at 2021-05-09 07:11:02
The latest post is submitted at 2021-05-09 07:14:35
The latest post is submitted at 2021-05-09 07:17:56
The latest post is submitted at 2021-05-09 07:21:22
The latest post is submitted at 2021-05-09 07:24:30
The latest post is submitted at 2021-05-09 07:28:03
The latest post is submitted at 2021-05-09 07:30:55
The latest post is submitted at 2021-05-09 07:34:37
The latest post is submitted at 2021-05-09 07:38:19
The latest post is submitted at 2021-05-09 07:42:03
The latest post is submitted at 2021-05-09 07:45:22
The latest post is submitted at 2021-05-09 07:48:36
The latest p

In [34]:
cur.execute('''SELECT subreddit,COUNT(*), COUNT(DISTINCT author) FROM Posts
                GROUP BY subreddit''')

print(cur.fetchall())

[('CryptoCurrency', 172461, 84948), ('GME', 145611, 40526), ('Superstonks', 146, 64), ('dogecoin', 576871, 169071), ('finance', 2941, 1267), ('options', 16589, 10251), ('pennystocks', 48514, 24938), ('stock', 100, 74), ('stocks', 23200, 13268), ('wallstreetbets', 954311, 499428)]


## Update the database by adding links.

In [4]:
cur.execute('''ALTER TABLE Posts
                ADD COLUMN ext_link TEXT''')


In [45]:
def dataUpdatePipeline(after, before, sub, conn):
    features = ['url', 'id']
    cursor = conn.cursor()
    while after < before:
        data = getPushShiftData(after, before, sub)
        if not data:
            print("There are no data anymore.")
            return 1
        for datum in data:
            cursor.execute('''UPDATE Posts
                                SET ext_link = ?
                                WHERE id = ? AND subreddit = ?'''
                              , (datum['url'], datum['id'], sub))
        
        after = getLatestTime(data) + 1
        conn.commit()
        print("The latest post is submitted at", dt.datetime.fromtimestamp(after-1))
        time.sleep(0.1)

def findStartingTime(cursor, subreddit):
    cursor.execute('''SELECT MAX(created) FROM Posts
                    WHERE subreddit = ? AND ext_link IS NOT NULL''', (subreddit,))
    datatimes = cursor.fetchone()
    
    return datatimes[0]

def getYourExistingSubs(cursor):
    cur.execute('''SELECT subreddit FROM Posts GROUP BY subreddit ORDER BY COUNT(*) ASC''')

    subreddits = list(map(lambda x: x[0], cur.fetchall()))
    
    return subreddits


In [46]:
#This chunk gives you all the subreddits you have scraped up to this point. Makes your life easier.
subreddits = getYourExistingSubs(cur)


In [40]:
for subreddit in subreddits:
    flag = 0
    print("Start working on {}".format(subreddit))
    end = int(time.time()-86400) #I subtracted by one day, so that we have some buffer.
    start = dt.datetime(2021,1,1).timestamp()

    dataTime = findStartingTime(cur,subreddit)
    if dataTime:
        start = max(start, dataTime)
    while True:
        try:
            dataUpdatePipeline(start, end, subreddit, conn)
            break
        except KeyboardInterrupt:
            print("Interrupted by keyboard. Stopping.")
            flag = 1
            break
        except:
            print("Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.")
            dataTime = findStartingTime(cur,subreddit)
            if dataTime:
                start = max(start, dataTime)
            time.sleep(1)
    
    if flag:
        break
            
            

Start working on Superstonks
The latest post is submitted at 2021-04-25 20:14:12
The latest post is submitted at 2021-05-09 10:09:15
There are no data anymore.
Start working on finance
The latest post is submitted at 2021-05-10 08:35:06
There are no data anymore.
Start working on options
The latest post is submitted at 2021-05-10 09:56:31
There are no data anymore.
Start working on pennystocks
The latest post is submitted at 2021-05-10 10:01:37
There are no data anymore.
Start working on GME
The latest post is submitted at 2021-05-09 19:10:47
The latest post is submitted at 2021-05-10 00:54:21
The latest post is submitted at 2021-05-10 07:37:07
The latest post is submitted at 2021-05-10 09:46:56
The latest post is submitted at 2021-05-10 10:01:04
There are no data anymore.
Start working on CryptoCurrency
The latest post is submitted at 2021-04-14 15:45:13
The latest post is submitted at 2021-04-14 16:57:51
The latest post is submitted at 2021-04-14 18:25:35
The latest post is submitted

In [37]:
cur.execute('''SELECT subreddit FROM Posts GROUP BY subreddit ORDER BY COUNT(*) ASC''')

subreddits = list(map(lambda x: x[0], cur.fetchall()))

subreddits

['Superstonks',
 'finance',
 'options',
 'pennystocks',
 'GME',
 'CryptoCurrency',
 'dogecoin',
 'wallstreetbets']

# Using PRAW to Download Additional Features

In this section, we will use praw to scrap post body, score, and upvote_ratio.

**Please use your own client_id, secret, etc. since we might be scraping at the same time. I don't want to get this account locked up.**


In [41]:
import praw

reddit = praw.Reddit(
    client_id="kxbUr-4PyE7DlQ",
    client_secret="Q5rIAPS9IHZ1QgOIkHNY09Y9VMxDsA",
    password="AACAXZDE",
    user_agent="testscript by u/kc_the_scraper",
    username="kc_the_scraper",
)

In [42]:
Sub = reddit.submission(id='eiev5d')
print(Sub.title, Sub.score, Sub.upvote_ratio)

#print(vars(Sub))

ben simmons needs a new team. 196 0.75


In [43]:
subreddit = 'options'
cur.execute('''SELECT id FROM Posts
                WHERE subreddit == ?
                LIMIT 10''', (subreddit,))

ids = cur.fetchall()


In [44]:
tStart = time.time()
for i,postId in enumerate(ids):
    Sub = reddit.submission(id=postId[0])
    print(Sub.selftext, Sub.score, Sub.upvote_ratio, Sub.url)
    

tEnd = time.time()

print(tEnd-tStart)

This morning I created a new scanner that alerts me when MACD gives a trend reversal signal and $STX was one of those that came up. I looked at chart, looks good. I go to buy call contracts to see what that looks like. Then I see every strike selling for .50+ but then there is one selling for .01 that went down -.50 that day even though the stock was having a good day. I'm just curious as to what happened here. I of course bought ASAP because I can get 50 contracts instead of 1 for the same price  and almost same OTM strike. (Expiration is Jan 08). Also, happy new years! 36 0.83 https://www.reddit.com/r/options/comments/ko6pbd/stx_69_call_dropped_from_60_premium_to_01_premium/
[removed] 1 1.0 https://www.reddit.com/r/options/comments/ko70kp/my_small_options_account/
[deleted] 5 0.74 https://www.reddit.com/r/options/comments/ko70sb/tax_considerations_for_assignment_on_ccs/
[removed] 1 1.0 https://www.reddit.com/r/options/comments/ko93x0/hk/
[removed] 1 1.0 https://www.reddit.com/r/optio

In [46]:
cur.execute('''CREATE TABLE IF NOT EXISTS PostBodyAndScore (
                    id TEXT PRIMARY KEY,
                    body TEXT,
                    score INT,
                    upvote_ratio FLOAT)''')

In [65]:
def timeLookupPRAW(cursor, subreddit):
    cursor.execute('''SELECT MIN(P.created), MAX(P.created)
                        FROM Posts P JOIN PostBodyAndScore B
                        ON P.id = B.id
                        WHERE subreddit = ?''', (subreddit,))
    return cursor.fetchone()

def retrieveID(cursor, num, subreddit, start, end):
    cursor.execute('''SELECT P.id
                        FROM (SELECT id, created FROM Posts
                                WHERE created >= ? AND created <= ? AND subreddit = ?) P LEFT JOIN PostBodyAndScore B
                        ON P.id = B.id
                        WHERE B.score IS NULL
                        ORDER BY P.created ASC
                        LIMIT ?''', (start,end, subreddit, num))
    
    return cursor.fetchall()

def storePostBodyAndScore(cursor, reddit, ids):
    for i,postId in enumerate(ids):
        Sub = reddit.submission(id=postId[0])
        cursor.execute('''INSERT OR IGNORE INTO PostBodyAndScore
                            VALUES (?,?,?,?)''', (postId[0],Sub.selftext,Sub.score,Sub.upvote_ratio))
        
    print("Finished adding post info for this batch.")
    conn.commit()

def fillOutTable(cursor, reddit, subreddit, start, end, num = 500):
    delta = 86400
    
    _, latest = timeLookupPRAW(cursor,subreddit)
    
    start = max(start,latest)
    
    while start < end:
        ids = retrieveID(cursor, num, subreddit, start, start + delta)
        print(ids)
        if not ids:
            start += delta
            print("==Finished all posts up to {}==".format(dt.datetime.fromtimestamp(start)))
            continue
        
        storePostBodyAndScore(cursor, reddit, ids)

    

In [66]:
subreddits = getYourExistingSubs(cur)
end = int(time.time()-86400)
start = dt.datetime(2021,1,1).timestamp()

for subreddit in subreddits:
    fillOutTable(cur, reddit, subreddit, start, end)


[]
==Finished all posts up to 2021-05-10 11:13:30==
[]
==Finished all posts up to 2021-05-10 09:31:19==
[('lg7dbe',), ('lg7dme',), ('lg7do5',), ('lg7emf',), ('lg7er1',), ('lg7er4',), ('lg7f35',), ('lg7glc',), ('lg7guh',), ('lg7h7d',), ('lg7hkm',), ('lg7i28',), ('lg7ii0',), ('lg7int',), ('lg7iw0',), ('lg7jev',), ('lg7k37',), ('lg7kdx',), ('lg7kwj',), ('lg7lv6',), ('lg7lw6',), ('lg7mng',), ('lg7mwp',), ('lg7myp',), ('lg7n31',), ('lg7nrc',), ('lg7nrt',), ('lg7o4m',), ('lg7o8y',), ('lg7obh',), ('lg7owb',), ('lg7owc',), ('lg7p6y',), ('lg7pau',), ('lg7pd7',), ('lg7ptl',), ('lg7pu8',), ('lg7qv7',), ('lg7qx9',), ('lg7r5q',), ('lg7rnb',), ('lg7ro3',), ('lg7rq6',), ('lg7rsv',), ('lg7s4t',), ('lg7sx4',), ('lg7t9x',), ('lg7ted',), ('lg7tks',), ('lg7unx',), ('lg7wf0',), ('lg7wqj',), ('lg7ww0',), ('lg7wwt',), ('lg7wx0',), ('lg7xif',), ('lg7xjo',), ('lg7xjp',), ('lg7y0t',), ('lg7ycw',), ('lg7yhy',), ('lg7yn9',), ('lg7yol',), ('lg7z32',), ('lg7zml',), ('lg80k8',), ('lg80w9',), ('lg8134',), ('lg81ij',)

KeyboardInterrupt: 